In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
#print(parent_dir)
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint


In [3]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.games import print_model as pm
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph

In [4]:
# setup
num = 50
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
#pm.print_model(model)

simple = SimpleSBML(model)
Reaction.reactions = []
Molecule.molecules = []
Reaction.initialize(simple)
SOM.initialize(Molecule.molecules)

In [5]:
Reaction.reactions

[v1: DFG -> E1; k1 * DFG,
 v2: DFG -> E2; k2 * DFG,
 v3: DFG -> Gly + Cn; k3 * DFG,
 v4: E1 -> Gly + _3DG; k4 * E1,
 v5: _3DG -> Cn; k5 * _3DG,
 v6: _3DG -> FA; k6 * _3DG,
 v7: E2 -> Gly + _1DG; k7 * E2,
 v8: _1DG -> Cn; k8 * _1DG,
 v9: _1DG -> AA; k9 * _1DG,
 v10: E1 -> Gly + Man; k10 * E1,
 v11: E1 -> Gly + Glu; k11 * E1,
 v12: Man -> Glu; k12 * Man,
 v13: Glu -> _3DG; k13 * Glu,
 v14: Gly + Cn -> Mel; k14 * Cn * Gly,
 v15: Cn -> AA + FA + MG; k15 * Cn,
 v16: E2 -> Gly + Fru; k16 * E2]

In [6]:
SOM.soms

[{DFG},
 {E1},
 {E2},
 {Gly},
 {Cn},
 {_3DG},
 {FA},
 {_1DG},
 {AA},
 {Man},
 {Glu},
 {Mel},
 {MG},
 {Fru}]

In [7]:
m = MESGraph(SOM.soms)

In [8]:
m.nodes

NodeView(({DFG}, {E1}, {E2}, {Gly}, {Cn}, {_3DG}, {FA}, {_1DG}, {AA}, {Man}, {Glu}, {Mel}, {MG}, {Fru}))

In [9]:
m.edges

OutEdgeView([])

In [10]:
if not m.edges:
    print("haha")

haha


In [11]:
# test constructor:
print(len(SOM.soms))
print(len(m.nodes) == 14)
print(len(m.edges) == 0)
ident = ""
for key, som in enumerate(SOM.soms):
    ident = ident + som.identifier
    if key < len(SOM.soms)-1:
        ident = ident + ";"
print(ident == m.identifier)

dfg = SOM.soms[0]
m.has_node(dfg)

14
True
True
True


True

In [60]:
print(Reaction.reactions)
unimulti_reaction = Reaction.reactions[2]
multiuni_reaction = Reaction.reactions[13]

[v1: DFG -> E1; k1 * DFG, v2: DFG -> E2; k2 * DFG, v3: DFG -> Gly + Cn; k3 * DFG, v4: E1 -> Gly + _3DG; k4 * E1, v5: _3DG -> Cn; k5 * _3DG, v6: _3DG -> FA; k6 * _3DG, v7: E2 -> Gly + _1DG; k7 * E2, v8: _1DG -> Cn; k8 * _1DG, v9: _1DG -> AA; k9 * _1DG, v10: E1 -> Gly + Man; k10 * E1, v11: E1 -> Gly + Glu; k11 * E1, v12: Man -> Glu; k12 * Man, v13: Glu -> _3DG; k13 * Glu, v14: Gly + Cn -> Mel; k14 * Cn * Gly, v15: Cn -> AA + FA + MG; k15 * Cn, v16: E2 -> Gly + Fru; k16 * E2]


In [40]:
print(uniuni_reaction)
print(uniuni_reaction1)
print(uniuni_reaction2)

v1: DFG -> E1; k1 * DFG
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG


In [14]:
unimulti_reaction

v3: DFG -> Gly + Cn; k3 * DFG

In [15]:
multiuni_reaction

v14: Gly + Cn -> Mel; k14 * Cn * Gly

In [16]:
m

{DFG};{E1};{E2};{Gly};{Cn};{_3DG};{FA};{_1DG};{AA};{Man};{Glu};{Mel};{MG};{Fru}

In [17]:
# test process uniuni reaction
m.processUniUniReaction(uniuni_reaction)
dfg_e1 = SOM.findSOM(list(dfg.molecules)[0])
m.has_node(dfg_e1) == True

True

In [62]:
multiuni_reaction

v14: Gly + Cn -> Mel; k14 * Cn * Gly

In [18]:
# test process unimulti reaction
m.processUniMultiReaction(unimulti_reaction)
prods = [SOM.findSOM(product.molecule) for product in unimulti_reaction.products]
for prod in prods:
    print(m.has_edge(prod, dfg_e1) == True)

True
True


In [19]:
# test process multiuni reaction
m.processMultiUniReaction(multiuni_reaction)
reacts = [SOM.findSOM(reactant.molecule) for reactant in multiuni_reaction.reactants]
mel = SOM.findSOM(Molecule.getMolecule('Mel'))
for react in reacts:
    print(m.has_edge(react, mel) == True)

True
True


In [31]:
# test addArc
source = [Molecule.getMolecule('Fru'), Molecule.getMolecule('Gly')]
destination = [Molecule.getMolecule('E2')]
dummy_reaction = Reaction.reactions[15]
m.addArc(source, destination, dummy_reaction)
print(m.has_edge(SOM.findSOM(source[0]), SOM.findSOM(destination[0])) == True)
print(m.has_edge(SOM.findSOM(source[1]), SOM.findSOM(destination[0])) == True)

True
True


In [44]:
# test checkTypeOneError
uniuni_reaction1 = Reaction.reactions[7]
uniuni_reaction2 = Reaction.reactions[8]
inequality_reaction1 = Reaction.reactions[14]
inequality_reaction2 = Reaction.reactions[15]
m.processUniUniReaction(uniuni_reaction1)
m.processUniUniReaction(uniuni_reaction2)
aa = Molecule.getMolecule('AA')
cn = Molecule.getMolecule('Cn')
mg = Molecule.getMolecule('MG')
print(m.checkTypeOneError((aa, cn), inequality_reaction1) == True)
print(m.checkTypeOneError((mg, aa), inequality_reaction2) == False)

We have Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA  <  Cn

True
True


In [59]:
# test analyze
k = m.analyze(Reaction.reactions)
print(k.identifier)
print(len(k.nodes) == 6)
print(len(k.edges) == 6)

We have Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA  <  Cn

We have Type I Error...
FA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA  <  Cn

{Gly}->{Mel}
{Gly}->{DFG, E1, E2}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}

True
True


In [37]:
m.analyze(Reaction.reactions)

We have Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA  <  Cn

We have Type I Error...
FA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA  <  Cn



{Gly}->{Mel}
{Gly}->{DFG, E1, E2}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}

In [63]:
dummy_reaction

v16: E2 -> Gly + Fru; k16 * E2

In [65]:
inequality_reaction2

v16: E2 -> Gly + Fru; k16 * E2